In [237]:
%matplotlib inline
import numpy as np
import sys
from bs4 import BeautifulSoup as BSoup
import requests

In [68]:
def profile_extractor(query):
    profile = {}
    r = requests.get('http://search.epfl.ch/psearch.action?request_locale=en&q=%s' %query)
    soup = BSoup(r.text.encode('utf-8'),'lxml')
    profile['Name'] = soup.find('div',class_='presentation').h4.string.replace('&nbsp;',' ')
    profile['Email'] = soup.find('div',class_='presentation').script.string.split('msgto(\'')[1].split('\'')[0]+'@epfl.ch'
    profile['SCIPER'] = soup.find('div',class_='button vcard').a['href'].split('id=')[1].split('&')[0]
    return profile

In [69]:
query = ' william trou'
for k,v in profile_extractor(query).items():
    print('%s' % v)

Farnood Salehi
farnood.salehi@epfl.ch
244595


In [136]:
#query = {'zz_x_UNITE_ACAD':'Architecture','ww_x_UNITE_ACAD':'942293','zz_x_PERIODE_ACAD':'2014-2015','zz_x_PERIODE_PEDAGO':'Bachelor+semestre+5','zz_x_HIVERETE':'Semestre+d%27automne'}
query = {'ww_x_UNITE_ACAD' :'944590'}
#r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270')
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&dummy=ok',params=query)
soup = BSoup(r.text.encode('utf-8'),'lxml')
print(r.url)

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&dummy=ok&ww_x_UNITE_ACAD=944590


In [138]:
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270')
soup = BSoup(r.text.encode('utf-8'),'lxml')
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 </head>
 <body>
  <div>
  </div>
  <title>
  </title>
  <script src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr" type="text/javascript">
  </script>
  <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css"/>
  <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator" rel="stylesheet" type="text/css"/>
  <div class="filtres">
   <form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
    <input name="ww_b_list" type="hidden" value="1"/>
    <input name="ww_i_reportmodel" type="hidden" value="133685247"/>
    <input name="ww_c_langue" type="hidden" value=""/>
    <h1 id="titre">
     Liste des Ã©tudiants inscrits par semestre
    </h1>
    <table border="0" id="format">
     <tr>
      <th>
       Format:
      </th>
     </tr>
     <tr>
      <td>
       <input checked="" name="ww_i_reportModelXsl" type="

In [348]:
links = soup.find_all("select")
#searched_word = 'Bachelor semestre 1'
#results = soup.body.find_all(string=re.compile('.*{0}.*'.format('Bachelor semestre 1')), recursive=True)
#print(results)

# The following code stores the values which are going to 
#be used in request.get for data

value_years = {} # Initilizing the datas
years = []
value_semesters = {}
semester = []
value_typeSemester = {}
value_typeSemester["Semestre d'automne"] = []
value_typeSemester['Semestre de printemps'] = []
typeSemester = []
typeSemester.append("Semestre d'automne")
typeSemester.append('Semestre de printemps')

["Semestre d'automne"]
['Semestre de printemps']
for i in range(2007,2017):
    value_years[str(i)] = []
    years.append(str(i))
for i in range(1,7):
    value_semesters['Bachelor semestre '+str(i)] = []
    semester.append('Bachelor semestre '+str(i))
value_semesters['Bachelor semestre '+str(5)+'b'] = []
value_semesters['Bachelor semestre '+str(6)+'b'] = []
semester.append('Bachelor semestre '+str(5)+'b')
semester.append('Bachelor semestre '+str(6)+'b')

for link in links:                     
    namesAll = link.find_all("option")
    for names in namesAll:
        #print(names.contents)
        if names.contents:
            if names.contents[0][:4] in years: # stores the values for years
                value_years[str(names.contents[0][:4])] = int(names['value'])
                print(names,names['value'],names.contents[0])
            elif names.contents[0] == 'Informatique': # stores the value for field Informatique
                value_informatiq = int(names['value'])
                print(names,names['value'],names.contents[0]) 
            elif names.contents[0] in semester: # stores the values for semesters
                value_semesters[names.contents[0]] = int(names['value'])
                print(names,names['value'],names.contents[0]) 
            elif names.contents[0] in typeSemester: # stores the values for type of semesters
                value_typeSemester[names.contents[0]] = int(names['value'])
                print(names,names['value'],names.contents[0]) 

<option value="249847">Informatique</option> 249847 Informatique
<option value="355925344">2016-2017</option> 355925344 2016-2017
<option value="213638028">2015-2016</option> 213638028 2015-2016
<option value="213637922">2014-2015</option> 213637922 2014-2015
<option value="213637754">2013-2014</option> 213637754 2013-2014
<option value="123456101">2012-2013</option> 123456101 2012-2013
<option value="123455150">2011-2012</option> 123455150 2011-2012
<option value="39486325">2010-2011</option> 39486325 2010-2011
<option value="978195">2009-2010</option> 978195 2009-2010
<option value="978187">2008-2009</option> 978187 2008-2009
<option value="978181">2007-2008</option> 978181 2007-2008
<option value="249108">Bachelor semestre 1</option> 249108 Bachelor semestre 1
<option value="249114">Bachelor semestre 2</option> 249114 Bachelor semestre 2
<option value="942155">Bachelor semestre 3</option> 942155 Bachelor semestre 3
<option value="942163">Bachelor semestre 4</option> 942163 Bachelor 

In [385]:
#query = {'zz_x_UNITE_ACAD':'Architecture','ww_x_UNITE_ACAD':'942293','zz_x_PERIODE_ACAD':'2014-2015','zz_x_PERIODE_PEDAGO':'Bachelor+semestre+5','zz_x_HIVERETE':'Semestre+d%27automne'}
query = {'ww_x_UNITE_ACAD' : str(value_informatiq),'ww_x_PERIODE_ACAD':str(value_years['2009']),
        'ww_x_PERIODE_PEDAGO':str(value_semesters['Bachelor semestre 4']),
         'ww_x_HIVERETE':str(value_typeSemester[typeSemester[1]])}

#r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270')
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?ww_x_GPS=-1&ww_i_reportmodel=133685247&ww_i_reportModelXsl=133685271',params=query)
soup = BSoup(r.text.encode('utf-8'),'lxml')
output = r.content

#output.write(resp.content)
#output.close()
#print(soup[0])

In [388]:
output

b'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><title></title></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><script>\n  function mailList(x) {\n   var vtop = (screen.height-200)/2;\n   var vleft=(screen.width-600)/2;\n   var w=open("", "emaillist", "Scrollbars=1,resizable=1,width=600,height=200,top="+vtop+",left="+vleft);\n   w.document.write(x);\n   w.document.close();\n   w.focus();\n  }\n </script><table><tr><th colspan="10">\n    Ces listes d\'\xe9tudiants ne seront d\xe9finitives qu\'\xe0 la mi-ao\xfbt une fois la commission d\'admission bachelor et la conf\xe9rence d\'examen pass\xe9es afin de s\'assurer que tous les \xe9tudiants nouvellement admis \xe0 l\'EPFL y figurent et que les \xe9tudiants en situation d\'\xe9chec d\xe9finitif soient ray\xe9s de ces listes\n   </th></tr><tr><th colspan="12"><font color="black">Informatique, 2009-2010, Bachelor semestre 4</font>\n&nbsp;(52

In [96]:
query = {'q':'Farnood Salehi'} 
profile = {}
r = requests.get('http://search.epfl.ch/psearch.action?request_locale=en&q=%s' ,params=query)
soup = BSoup(r.text.encode('utf-8'),'lxml')
profile['Name'] = soup.find('div',class_='presentation').h4.string.replace('&nbsp;',' ')
profile['Email'] = soup.find('div',class_='presentation').script.string.split('msgto(\'')[1].split('\'')[0]+'@epfl.ch'
profile['SCIPER'] = soup.find('div',class_='button vcard').a['href'].split('id=')[1].split('&')[0]
#soup

In [187]:
query = {'ww_x_UNITE_ACAD' :'944590'}
#r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270')
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&dummy=ok',params=query)
soup = BSoup(r.text.encode('utf-8'),'lxml')
print(r.url)

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&dummy=ok&ww_x_UNITE_ACAD=944590
